In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Prepare Batch

In [2]:
import os
import torch
import pandas as pd
from recfldgrn.datapoint import load_df_data_from_folder
from fieldnn.utils.layerfn import traverse, convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PN@DT-DTDftGrn',
 'PNSect@SectName-PNSctNmDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn',
 'Smoking@DT-DTDftGrn',
 'Smoking@V-SmokingDftGrn']

In [3]:
recfldgrn_list =['P@age-AgeNumeDftGrn',
                 'P@basicInfo-basicInfoDftGrn',
                 
                 'EC@BasicInfo-BasicDftGrn',
                 'EC@DT_min-DTDftGrn',
                 
                 'A1C@DT-DTDftGrn',
                 'A1C@V-A1CNumeDftGrn',
                 
                 'Diag@DT-DTDftGrn',
                 'Diag@Value-DiagDftGrn',
                 
                 'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [4]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P@basicInfo-basicInfoDftGrn_idx (4, 2)
B-P-EC@BasicInfo-BasicDftGrn_idx (4, 25, 2)
B-P-EC@DT_min-DTDftGrn_idx (4, 25, 7)
B-P-EC-A1C@DT-DTDftGrn_idx (4, 25, 1, 7)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


In [5]:
from fieldnn.sublayer.expander import Expander_Layer
from fieldnn.configfn.expanderfn import get_expander_para

full_recfldgrn_list = [
                  i for i in batch_rfg
                ]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P@basicInfo-basicInfoDftGrn_idx',
 'B-P-EC@BasicInfo-BasicDftGrn_idx',
 'B-P-EC@DT_min-DTDftGrn_idx',
 'B-P-EC-A1C@DT-DTDftGrn_idx',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [6]:
full_rfg_to_basicInfo = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) full recfldgrn information
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    recfldgrn = rec + '@' + fld + '-' + grn

    # (2) get vocab information
    fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
    df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

    # no matter what type of grain, in the end, we will have vocab_tokenizer.
    if 'LLM' in full_recfldgrn:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # vocab_size = len(vocab_tokenizer)
        # print(vocab_size)
        init = vocab_tokenizer.name_or_path
    else:
        vocab_tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # vocab_size = len(vocab_tokenizer)
        # print(vocab_size)
        init = 'random'
    
    d = {'vocabsize_tokenizer': vocab_tokenizer, 'init': init}
    full_rfg_to_basicInfo[full_recfldgrn] = d
    
    
len(full_rfg_to_basicInfo)

9

In [7]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


full_rfg_to_ExpanderConfig = {}

for full_recfldgrn in full_recfldgrn_list:
    
    # determine the nn_name
    if '_idx' in full_recfldgrn and 'LLM' not in full_recfldgrn:
        nn_name = 'CateEmbed'

    elif '_idx' in full_recfldgrn and 'LLM' in full_recfldgrn:
        nn_name = 'LLMEmbed'

    elif '_wgt' in full_recfldgrn:
        nn_name = 'NumeEmbed'
    else:
        raise ValueError(f'full_recfldgrn is not correct "{full_recfldgrn}"')
    
    d = full_rfg_to_basicInfo[full_recfldgrn]
    
    # print('\n----')
    # print(full_recfldgrn)
    # print(d)
    # print(nn_name)
    expander_para = get_expander_para(full_recfldgrn, 
                                       nn_name,
                                       embed_size, d['vocabsize_tokenizer'], d['init'], 
                                       process)
    # print(expander_para)
    output_recfld = full_recfldgrn.split('Grn')[0]
    
    full_rfg_to_ExpanderConfig[full_recfldgrn] = {
        'full_recfldgrn': full_recfldgrn,
        'output_recfld': output_recfld,
        'expander_para': expander_para,
    }
    
    
# full_rfg_to_ExpanderConfig

In [8]:
FullRFG_2_Embed = {}

for full_recfldgrn, ExpanderConfig in full_rfg_to_ExpanderConfig.items():
    
    FullRFG_2_Embed[full_recfldgrn] = Expander_Layer(**ExpanderConfig)
    print(full_recfldgrn)
    print(FullRFG_2_Embed[full_recfldgrn])

B-P@age-AgeNumeDftGrn_wgt
Expander_Layer(
  (Embed): NumeEmbeddingLayer(
    (embedding): Embedding(19, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P@basicInfo-basicInfoDftGrn_idx
Expander_Layer(
  (Embed): CateEmbeddingLayer(
    (embedding): Embedding(7, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC@BasicInfo-BasicDftGrn_idx
Expander_Layer(
  (Embed): CateEmbeddingLayer(
    (embedding): Embedding(9, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC@DT_min-DTDftGrn_i

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
Expander_Layer(
  (Embed): LLMEmbeddingLayer(
    (LLM): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense):

In [9]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print(full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str: 
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')

    print('\n')

B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information


B-P@basicInfo-basicInfoDftGrn_idx
(4, 2)
torch.Size([4, 2]) <------ holder shape
Empty <------ holder wgt information


B-P-EC@BasicInfo-BasicDftGrn_idx
(4, 25, 2)
torch.Size([4, 25, 2]) <------ holder shape
Empty <------ holder wgt information


B-P-EC@DT_min-DTDftGrn_idx
(4, 25, 7)
torch.Size([4, 25, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-A1C@DT-DTDftGrn_idx
(4, 25, 1, 7)
torch.Size([4, 25, 1, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


B-P-EC-Diag@DT-DTDftGrn_idx
(4, 25, 22, 7)
torch.Size([4, 25, 22, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder sha

In [10]:
full_recfldgrn_EmbedTensor = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print('Input:',full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str:  
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')
    
    NN = FullRFG_2_Embed[full_recfldgrn] 
    print(holder.shape, '<----- holder shape')
    output_recfld, holder, info = NN(full_recfldgrn, holder, holder_wgt)
    print('Output:', output_recfld)
    print(info.shape, '<----- info shape')
    full_recfldgrn_EmbedTensor[output_recfld] = {'holder': holder, 'info': info}
    print('\n')

Input: B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information
torch.Size([4, 19]) <----- holder shape
Output: B-P@age-AgeNumeDft
torch.Size([4, 19, 128]) <----- info shape


Input: B-P@basicInfo-basicInfoDftGrn_idx
(4, 2)
torch.Size([4, 2]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 2]) <----- holder shape
Output: B-P@basicInfo-basicInfoDft
torch.Size([4, 2, 128]) <----- info shape


Input: B-P-EC@BasicInfo-BasicDftGrn_idx
(4, 25, 2)
torch.Size([4, 25, 2]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 2]) <----- holder shape
Output: B-P-EC@BasicInfo-BasicDft
torch.Size([4, 25, 2, 128]) <----- info shape


Input: B-P-EC@DT_min-DTDftGrn_idx
(4, 25, 7)
torch.Size([4, 25, 7]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 7]) <----- holder shape
Output: B-P-EC@DT_min-DTDft
torch.Size([4, 25, 7, 128]) <----- info shape


Input: B-P

# Get the Starting Tensor and Tensor Name

In [11]:
# full_recfldgrn_EmbedTensor = {}
RECFLD_TO_TENSOR = {}

for full_recfld, info_dict in full_recfldgrn_EmbedTensor.items():
    print(full_recfld, info_dict['info'].shape)
    RECFLD_TO_TENSOR[full_recfld] = info_dict

B-P@age-AgeNumeDft torch.Size([4, 19, 128])
B-P@basicInfo-basicInfoDft torch.Size([4, 2, 128])
B-P-EC@BasicInfo-BasicDft torch.Size([4, 25, 2, 128])
B-P-EC@DT_min-DTDft torch.Size([4, 25, 7, 128])
B-P-EC-A1C@DT-DTDft torch.Size([4, 25, 1, 7, 128])
B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@DT-DTDft torch.Size([4, 25, 22, 7, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


In [12]:
[i for i in RECFLD_TO_TENSOR]

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [13]:
max_layer = max([len(i.split('-')) for i in RECFLD_TO_TENSOR])
max_layer

7

In [17]:
from fieldnn.utils.dataflowfn import get_dataflow_table
from fieldnn.utils.dataflowfn import update_df_dataflow, update_df_dataflow_completename

In [18]:
full_recfldgrn_list = [i for i in RECFLD_TO_TENSOR]
full_recfldgrn_list

['B-P@age-AgeNumeDft',
 'B-P@basicInfo-basicInfoDft',
 'B-P-EC@BasicInfo-BasicDft',
 'B-P-EC@DT_min-DTDft',
 'B-P-EC-A1C@DT-DTDft',
 'B-P-EC-A1C@V-A1CNumeDft',
 'B-P-EC-Diag@DT-DTDft',
 'B-P-EC-Diag@Value-DiagDft',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM']

In [19]:
df_dataflow = get_dataflow_table(full_recfldgrn_list)
df_dataflow

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,NaN
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,NaN
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,NaN,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,NaN,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,NaN,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,NaN,NaN,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,NaN,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,NaN,NaN,NaN
PNSectSent@Sentence-Tk@TknzLLM,B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM,NaN,NaN,NaN,NaN,NaN


In [20]:
df_dataflow_new = update_df_dataflow(df_dataflow, style = 'Reducer&Merger')
df_dataflow_new

,7,6,5,4,3,2
recfldgrn,,,,,,
P@age-AgeNumeDft,NaN,NaN,NaN,NaN,B-P@age-AgeNumeDft,B-P@age
P@basicInfo-basicInfoDft,NaN,NaN,NaN,NaN,B-P@basicInfo-basicInfoDft,B-P@basicInfo
EC@BasicInfo-BasicDft,NaN,NaN,NaN,B-P-EC@BasicInfo-BasicDft,B-P-EC@BasicInfo,NaN
EC@DT_min-DTDft,NaN,NaN,NaN,B-P-EC@DT_min-DTDft,B-P-EC@DT_min,NaN
A1C@DT-DTDft,NaN,NaN,B-P-EC-A1C@DT-DTDft,B-P-EC-A1C@DT,NaN,NaN
A1C@V-A1CNumeDft,NaN,NaN,B-P-EC-A1C@V-A1CNumeDft,B-P-EC-A1C@V,NaN,NaN
(Merge)A1C@DT&V,NaN,NaN,NaN,B-P-EC-A1C,B-P-EC@A1C,NaN
Diag@DT-DTDft,NaN,NaN,B-P-EC-Diag@DT-DTDft,B-P-EC-Diag@DT,NaN,NaN
Diag@Value-DiagDft,NaN,NaN,B-P-EC-Diag@Value-DiagDft,B-P-EC-Diag@Value,NaN,NaN


# Get Layer Neural Name

In [24]:
df_dataflow = df_dataflow_new

In [25]:
from fieldnn.utils.dataflowfn import get_SubUnit_List

In [26]:
df_SubUnit = get_SubUnit_List(df_dataflow)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3


# Generation the SubUnit Config 

In [27]:
def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for NAME in SubUnitName:
        if NAME == 'R':
            nn_type = 'reducer'
        elif NAME == 'M':
            nn_type = 'merger'
        elif NAME == 'L':
            nn_type = 'rearner'
        elif NAME == 'E':
            nn_type = 'expander'
        nn_name = default_BasicNNtype_To_NNName[nn_type]
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList
    
    
############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    'expander': None, # TODO: ignore this currently
}
#############################################


SubUnitName = 'R'
SubUnitNNList = mapping_SubUnitName_to_SubUnitNNList(SubUnitName, default_BasicNNtype_To_NNName)
SubUnitNNList

['reducer-Max']

In [28]:
df_SubUnit['SubUnit_BasicNN_List'] = df_SubUnit['SubUnitName'].apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x, default_BasicNNtype_To_NNName))
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,[reducer-Max]
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,[reducer-Max]
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,[reducer-Max]
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,[reducer-Max]
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,[reducer-Max]
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,[reducer-Max]
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,[reducer-Max]
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max]"
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max]"
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,[reducer-Max]


## BasicNN: Reducer

In [29]:
def get_reducer_para(nn_name, nn_para, input_size, output_size, postprocess):
    # reducer is type of BasicNN.
    reducer_layer_para = {}
    reducer_layer_para['nn_type'] = 'reducer'
    reducer_layer_para['nn_name'] = nn_name
    reducer_layer_para['nn_para'] = nn_para
    reducer_layer_para['input_size'] = input_size
    reducer_layer_para['output_size'] = output_size
    reducer_layer_para['postprocess'] = postprocess
    return reducer_layer_para

### Focus on One SubUnit

In [30]:
SubUnit_info = df_SubUnit.iloc[0]
SubUnit_info

SubUnitName                                                     R
input_names             [B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk]
output_name                           B-P-EC-PN-PNSect-PNSectSent
input_layerid                                                   7
output_layerid                                                  6
SubUnit_BasicNN_List                                [reducer-Max]
Name: 0, dtype: object

In [31]:
SubUnitName = SubUnit_info['SubUnitName'] 
# SubUnitName = SubUnit_info['SubUnitName'] 
# here we want to generate the para_list. 
SubUnit_input_names = SubUnit_info['input_names']
SubUnit_output_name = SubUnit_info['output_name']
SubUnit_input_layerid = SubUnit_info['input_layerid']
SubUnit_output_layerid = SubUnit_info['output_layerid']

print(SubUnit_input_names, '<--- SubUnit_input_names')
print(SubUnit_output_name, '<--- SubUnit_output_name')

['B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk'] <--- SubUnit_input_names
B-P-EC-PN-PNSect-PNSectSent <--- SubUnit_output_name


In [32]:
SubUnit_BasicNN_List = SubUnit_info['SubUnit_BasicNN_List']
SubUnit_BasicNN_List

['reducer-Max']

In [33]:
# One SubUnit would contains multiple Basic NN
# currently, we focus on one specific NN - reducer's Max
# prepare your input here
############################ Hyperparameters
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
default_nnpara = {}
############################

In [34]:
BasicNN_idx = 0

# Next Time we will loop this
#### Getting the BasicNN information.
nn_type_nn_name = SubUnit_BasicNN_List[BasicNN_idx]

nn_type, nn_name = nn_type_nn_name.split('-')
# Image here you are in the loop already.
# Current Iteration is 0

# actually, the following code only deals with nn_type == 'reducer'
# We will add the following condition in the whole loop.
# if nn_type == 'reducer': 
assert len(SubUnit_input_names) == 1

# Get the input_names_nnlvl
input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration

# Get the output_name_nnlvl
output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)


# Prepare the para for the NN layer
nn_para = default_nnpara # this will be updated.

# Get the input_size
input_size = embed_size

# Get the output_size
if nn_name.lower() == 'concat':
    # this types of merger only happened in the R in MLRL. 
    # you can skip this part safely if you haven't encountered M. 
    fld_childflds = input_names_nnlvl[0].split('-')[-1]
    assert '@' in fld_childflds
    childflds = fld_childflds.split('@')[-1]
    childflds = childflds.split('&')
    output_size = len(childflds) * embed_size
else:
    # usually the most case. 
    output_size = embed_size 

# Get the postprocess
postprocess = process

# Derive the para
para = get_reducer_para(nn_name, nn_para, input_size, output_size, postprocess)  

# have a look at the para here. 
print(input_names_nnlvl, '<--------- input_names_nnlvl')
print(output_name_nnlvl, '<--------- output_name_nnlvl')
para



SubUnit_Config = {'input_names_nnlvl': input_names_nnlvl, 
                  'output_name_nnlvl': output_name_nnlvl, 
                  'para': para}




['B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk'] <--------- input_names_nnlvl
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk <--------- output_name_nnlvl


In [35]:
# write it as a function

# Convert the above block to a function

def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    # Image here you are in the loop already.
    # Current Iteration is 0

    # actually, the following code only deals with nn_type == 'reducer'
    # We will add the following condition in the whole loop.
    if nn_type == 'reducer': 
        # assert len(SubUnit_input_names) == 1
        # Get the input_names_nnlvl
        # input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration

        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)

        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size

        # Get the output_size
        if nn_name.lower() == 'concat':
            # this types of merger only happened in the R in MLRL. 
            # you can skip this part safely if you haven't encountered M. 
            fld_childflds = input_names_nnlvl[0].split('-')[-1]
            assert '@' in fld_childflds
            childflds = fld_childflds.split('@')[-1]
            childflds = childflds.split('&')
            output_size = len(childflds) * embed_size
        else:
            # usually the most case. 
            output_size = embed_size 

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, nn_para, input_size, output_size, postprocess)  
        
    else:
        raise ValueError('nn_type')

    # have a look at the para here. 
    # print(input_names_nnlvl, '<--------- input_names_nnlvl')
    # print(output_name_nnlvl, '<--------- output_name_nnlvl')
    # para

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                      'output_name_nnlvl': output_name_nnlvl, 
                      f'{nn_type}_para': para}
    
    return Basic_Config



In [36]:
SubUnit_input_names

['B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk']

In [37]:
# the question is how to create the SubUnit_BasicNN_List for each SubUnit
# we will turn to this later. 
# print(SubUnit_BasicNN_List)

SubUnit_BasicNN_Config_List = []
for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

    # Get the input_names_nnlvl
    if basic_nn_idx == 0:
        input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
    else:
        input_names_nnlvl = output_name_nnlvl
        
    Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                           input_names_nnlvl, 
                                           default_nnpara, 
                                           embed_size, 
                                           process)
    output_name_nnlvl = Basic_Config['output_name_nnlvl']
    BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
    SubUnit_BasicNN_Config_List.append(BasicNN_Config)
    
    ## TODO
    # also check the input_size of dim and output_size of dim
    
final_output_name_nnlvl = output_name_nnlvl


print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
print(SubUnit_output_name, '<------- SubUnit_output_name')
SubUnit_BasicNN_Config_List

B-P-EC-PN-PNSect-PNSectSent@Sentence@Tk <------- final_output_name_nnlvl
B-P-EC-PN-PNSect-PNSectSent <------- SubUnit_output_name


[{'nn_type_nn_name': 'reducer-Max',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk'],
   'output_name_nnlvl': 'B-P-EC-PN-PNSect-PNSectSent@Sentence@Tk',
   'reducer_para': {'nn_type': 'reducer',
    'nn_name': 'Max',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}}]

## BasicNN: Merger

In [38]:
def get_merger_para(nn_name, nn_para, input_size, output_size, postprocess):
    para = {}
    para['nn_type'] = 'merger'
    para['nn_name'] = nn_name
    para['nn_para'] = nn_para
    para['input_size'] = input_size
    para['output_size'] = output_size
    para['postprocess'] = postprocess
    return para

### Focus on One SubUnit

In [39]:
# df_SubUnit

In [40]:
SubUnit_info = df_SubUnit.iloc[7]
SubUnit_info

SubUnitName                                        MR
input_names             [B-P-EC-A1C@DT, B-P-EC-A1C@V]
output_name                                B-P-EC-A1C
input_layerid                                       4
output_layerid                                      4
SubUnit_BasicNN_List     [merger-Merger, reducer-Max]
Name: 7, dtype: object

In [41]:
SubUnitName = SubUnit_info['SubUnitName'] 
# SubUnitName = SubUnit_info['SubUnitName'] 
# here we want to generate the para_list. 
SubUnit_input_names = SubUnit_info['input_names']
SubUnit_output_name = SubUnit_info['output_name']
SubUnit_input_layerid = SubUnit_info['input_layerid']
SubUnit_output_layerid = SubUnit_info['output_layerid']

print(SubUnit_input_names, '<--- SubUnit_input_names')
print(SubUnit_output_name, '<--- SubUnit_output_name')

['B-P-EC-A1C@DT', 'B-P-EC-A1C@V'] <--- SubUnit_input_names
B-P-EC-A1C <--- SubUnit_output_name


In [42]:
SubUnit_BasicNN_List = SubUnit_info['SubUnit_BasicNN_List']
SubUnit_BasicNN_List

['merger-Merger', 'reducer-Max']

In [46]:
# write it as a function

# Convert the above block to a function

def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    # Image here you are in the loop already.
    # Current Iteration is 0

    # actually, the following code only deals with nn_type == 'reducer'
    # We will add the following condition in the whole loop.
    if nn_type == 'reducer': 
        # assert len(SubUnit_input_names) == 1
        # Get the input_names_nnlvl
        # input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)

        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size

        # Get the output_size
        if nn_name.lower() == 'concat':
            # this types of merger only happened in the R in MLRL. 
            # you can skip this part safely if you haven't encountered M. 
            fld_childflds = input_names_nnlvl[0].split('-')[-1]
            assert '@' in fld_childflds
            childflds = fld_childflds.split('@')[-1]
            childflds = childflds.split('&')
            output_size = len(childflds) * embed_size
        else:
            # usually the most case. 
            output_size = embed_size 

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, nn_para, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        fld_childflds = '&'.join(childflds)
        output_prefix = input_names_nnlvl[0].replace('-' + childflds[0], '')
        output_name_nnlvl = output_prefix + '@' + fld_childflds
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, nn_para, input_size, output_size, postprocess) 
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    # have a look at the para here. 
    # print(input_names_nnlvl, '<--------- input_names_nnlvl')
    # print(output_name_nnlvl, '<--------- output_name_nnlvl')
    # para

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                      'output_name_nnlvl': output_name_nnlvl, 
                      f'{nn_type}_para': para}
    
    return Basic_Config


In [44]:
SubUnit_BasicNN_List

['merger-Merger', 'reducer-Max']

In [45]:
# the question is how to create the SubUnit_BasicNN_List for each SubUnit
# we will turn to this later. 
# print(SubUnit_BasicNN_List)

SubUnit_BasicNN_Config_List = []
for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

    # Get the input_names_nnlvl
    if basic_nn_idx == 0:
        input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
    else:
        input_names_nnlvl = [output_name_nnlvl]
        
    print(nn_type_nn_name)
    Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                           input_names_nnlvl, 
                                           default_nnpara, 
                                           embed_size, 
                                           process)
    output_name_nnlvl = Basic_Config['output_name_nnlvl']
    BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
    SubUnit_BasicNN_Config_List.append(BasicNN_Config)
    
    ## TODO
    # also check the input_size of dim and output_size of dim
    
final_output_name_nnlvl = output_name_nnlvl


print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
print(SubUnit_output_name, '<------- SubUnit_output_name')
SubUnit_BasicNN_Config_List

merger-Merger
reducer-Max
B-P@EC@A1C@DT&A1C@V <------- final_output_name_nnlvl
B-P-EC-A1C <------- SubUnit_output_name


[{'nn_type_nn_name': 'merger-Merger',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC-A1C@DT', 'B-P-EC-A1C@V'],
   'output_name_nnlvl': 'B-P-EC@A1C@DT&A1C@V',
   'merger_para': {'nn_type': 'merger',
    'nn_name': 'Merger',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}},
 {'nn_type_nn_name': 'reducer-Max',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC@A1C@DT&A1C@V'],
   'output_name_nnlvl': 'B-P@EC@A1C@DT&A1C@V',
   'reducer_para': {'nn_type': 'reducer',
    'nn_name': 'Max',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}}]

In [47]:
# write the above block to the python function

In [62]:
# the question is how to create the SubUnit_BasicNN_List for each SubUnit
# we will turn to this later. 
# print(SubUnit_BasicNN_List)

def get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                    SubUnit_input_names, 
                                    SubUnit_output_name, 
                                    default_nnpara, 
                                    embed_size, 
                                    process
                                   ):
    
    
    # TODO: also add the layer_idx in order to deal with the learn_layer_para.
    # This function also needs to be updated. 
    
    SubUnit_BasicNN_Config_List = []
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):

        # Get the input_names_nnlvl
        if basic_nn_idx == 0:
            input_names_nnlvl = SubUnit_input_names # this assigments only works for the first iteration
        else:
            input_names_nnlvl = [output_name_nnlvl]

        # print(nn_type_nn_name)
        Basic_Config = generate_BasicNN_Config(nn_type_nn_name, 
                                               input_names_nnlvl, 
                                               default_nnpara, 
                                               embed_size, 
                                               process)
        output_name_nnlvl = Basic_Config['output_name_nnlvl']
        BasicNN_Config = {'nn_type_nn_name': nn_type_nn_name, 'Basic_Config': Basic_Config}
        SubUnit_BasicNN_Config_List.append(BasicNN_Config)
        
        # also check the input_size of dim and output_size of dim

    final_output_name_nnlvl = output_name_nnlvl

    # print(final_output_name_nnlvl, '<------- final_output_name_nnlvl')
    # print(SubUnit_output_name, '<------- SubUnit_output_name')
    assert SubUnit_output_name in final_output_name_nnlvl
    return SubUnit_BasicNN_Config_List

In [63]:
SubUnit_BasicNN_Config_List = get_SubUnit_BasicNN_Config_List(SubUnit_BasicNN_List, 
                                                                SubUnit_input_names, 
                                                                SubUnit_output_name, 
                                                                default_nnpara, 
                                                                embed_size, 
                                                                process
                                                               )
SubUnit_BasicNN_Config_List

[{'nn_type_nn_name': 'merger-Merger',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC-A1C@DT', 'B-P-EC-A1C@V'],
   'output_name_nnlvl': 'B-P-EC@A1C@DT&A1C@V',
   'merger_para': {'nn_type': 'merger',
    'nn_name': 'Merger',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}},
 {'nn_type_nn_name': 'reducer-Max',
  'Basic_Config': {'input_names_nnlvl': ['B-P-EC@A1C@DT&A1C@V'],
   'output_name_nnlvl': 'B-P@EC@A1C@DT&A1C@V',
   'reducer_para': {'nn_type': 'reducer',
    'nn_name': 'Max',
    'nn_para': {},
    'input_size': 128,
    'output_size': 128,
    'postprocess': {'activator': 'gelu',
     'dropout': {'p': 0.5, 'inplace': False},
     'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}}}}]

In [64]:
print(default_nnpara)
print(embed_size)
print(process)

{}
128
{'activator': 'gelu', 'dropout': {'p': 0.5, 'inplace': False}, 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}


In [65]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               x['output_name'],
                                                               default_nnpara, 
                                                               embed_size, 
                                                               process
                                                              ),
                     axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_BasicNN_Config_List
0,R,[B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk],B-P-EC-PN-PNSect-PNSectSent,7,6,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
1,R,[B-P-EC-PN-PNSect-PNSectSent],B-P-EC-PN-PNSect,6,5,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
2,R,[B-P-EC-A1C@DT-DTDft],B-P-EC-A1C@DT,5,4,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
3,R,[B-P-EC-A1C@V-A1CNumeDft],B-P-EC-A1C@V,5,4,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
4,R,[B-P-EC-Diag@DT-DTDft],B-P-EC-Diag@DT,5,4,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
5,R,[B-P-EC-Diag@Value-DiagDft],B-P-EC-Diag@Value,5,4,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
6,R,[B-P-EC-PN-PNSect],B-P-EC-PN,5,4,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
7,MR,"[B-P-EC-A1C@DT, B-P-EC-A1C@V]",B-P-EC-A1C,4,4,"[merger-Merger, reducer-Max]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
8,MR,"[B-P-EC-Diag@DT, B-P-EC-Diag@Value]",B-P-EC-Diag,4,4,"[merger-Merger, reducer-Max]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."
9,R,[B-P-EC@BasicInfo-BasicDft],B-P-EC@BasicInfo,4,3,[reducer-Max],"[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."


## TODO: BasicNN: Learner

## TODO: BasicNN: Expander (Embed)

# SubUnit (1st Version)

Connect All the NN

## Module

In [2]:
import torch

class SubUnit_Layer(torch.nn.Module):
    '''Currently, it is not latest version'''
    
    def __init__(self, SubUnit_info):
        super(SubUnit_Layer, self).__init__()
        
        # the input names for the SubUnit
        self.SubUnit_input_names = SubUnit_info['input_names']
        
        # the output name for this SubUnit
        self.SubUnit_output_name = SubUnit_info['output_name']
        
        # get the SubUnit's BasicNN Config List
        self.SubUnit_BasicNN_Config_List = SubUnit_info['SubUnit_BasicNN_Config_List']
        
        
        # construct the LayersDict to hold all BasicNN within this SubUnit.
        self.LayersDict = torch.nn.ModuleDict()
        
        # initialize all the BasicNN for this SubUnit.
        for idx, BasicNN_Config_Dict in enumerate(self.SubUnit_BasicNN_Config_List):
            
            # nn_type_nn_name: like reducer-Max, learner-TFM, merger-Merger, expander-llmembed.
            nn_type_nn_name = BasicNN_Config_Dict['nn_type_nn_name']
            
            # Basic_Config: the Config for this NN.
            Basic_Config = BasicNN_Config_Dict['Basic_Config']
            
            input_names_nnlvl = Basic_Config['input_names_nnlvl']
            output_name_nnlvl = Basic_Config['output_name_nnlvl']
                
            if 'expander' in nn_type_nn_name:
                expander_para = Basic_Config['expander_para']
                NN = Expander_Layer(input_names_nnlvl, output_name_nnlvl, expander_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            elif 'reducer' in nn_type_nn_name:
                reducer_para = Basic_Config['reducer_para']
                NN = Reducer_Layer(input_names_nnlvl, output_name_nnlvl, reducer_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            elif 'merger' in nn_type_nn_name:
                merger_para = Basic_Config['merger_para']
                NN = Merger_Layer(input_names_nnlvl, output_name_nnlvl, merger_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
                
            elif 'learner' in nn_type_nn_name:
                learner_para = Basic_Config['learner_para']
                NN = Learner_Layer(input_names_nnlvl, output_name_nnlvl, learner_para)
                self.LayersDict[f'{idx}_{nn_type_nn_name}'] = NN
                
            else:
                raise ValueError(f'Current BasicNN {nn_type_nn_name} is not available')


    def forward(self, SubUnit_input_names, RECFLD_TO_TENSOR):

        INPUTS_TO_INFODICT = {}
        
        for idx, BasicNN_Config_Dict in enumerate(self.SubUnit_BasicNN_Config_List):
            
            nn_type_nn_name = BasicNN_Config_Dict['nn_type_nn_name']
            Basic_Config = BasicNN_Config_Dict['Basic_Config']
            input_names_nnlvl = Basic_Config['input_names_nnlvl']
            output_name_nnlvl = Basic_Config['output_name_nnlvl']
            
            NN = self.LayersDict[f'{idx}_{nn_type_nn_name}']
            
            # prepare the input.
            if idx == 0:
                for tensor_name in input_names_nnlvl:
                    INPUTS_TO_INFODICT[tensor_name] = RECFLD_TO_TENSOR[tensor_name]
            else:
                assert input_names_nnlvl in INPUTS_TO_INFODICT
                
            output_name_nnlvl, info_dict = NN(input_names_nnlvl, INPUTS_TO_INFODICT)
            
            # current output will be the input in the next round. 
            INPUTS_TO_INFODICT[output_name_nnlvl] = info_dict
            
        # pick up the SubUnit_output_name and its info_dict
        final_output_name_nnlvl = output_name_nnlvl
        SubUnit_output_name = self.SubUnit_output_name
        if not SubUnit_output_name in final_output_name_nnlvl:
            print(f'Mismatched SubUnit Output and Final NN Output: {final_output_name_nnlvl} vs {output_name_nnlvl}')
        
        info_dict = INPUTS_TO_INFODICT[final_output_name_nnlvl]
        
        return SubUnit_output_name, info_dict
    